In [23]:
class VenueInfo:
    def __init__(self, org_type, category_id, category, title, address, city, county, lo, la, open_time, pic):
        self.org_type = org_type
        self.category_id = category_id
        self.category = category
        self.title = title
        self.address = address
        self.city = city
        self.county = county
        self.longitude = lo
        self.latitude = la
        self.open_time = open_time
        self.picture = pic

    def __str__(self):
        # 使用 __dict__ 将对象的属性转换为字典
        # 然后将字典转换为 JSON 字符串
        return json.dumps(self.__dict__, ensure_ascii=False, indent=4)

    def to_dict(self):
        # 转换为字典
        return self.__dict__

In [32]:
import os
import pandas as pd
import requests

# 读取 Excel 文件
excel_file = '/Users/pangmengting/Downloads/文化场馆表-2.xlsx'
# 从 1560 行开始
df = pd.read_excel(excel_file, engine='openpyxl', skiprows=range(1, 1561))

# 创建保存图片的文件夹
# output_folder = 'images'
# os.makedirs(output_folder, exist_ok=True)
venues = []
# 遍历每一行,下载图片并用标题命名
for index, row in df.iterrows():
    org_type = row['org_type']
    category_id = row['分类ID']
    category = row['分类名称']
    title = row['名称']
    address = row['地址']
    city = row['城市']
    county = row['区县']
    lo = row['经度']
    la = row['纬度']
    open_time = row['开放时间']
    pic = row['图片']

    if pd.isna(pic) or pic == '':
        # 如果图片字段是空值或者空字符串，则跳过当前循环
        continue

    venue = VenueInfo(
        org_type=org_type,
        category_id=category_id,
        category=category,
        title=title,
        address=address,
        city=city,
        county=county,
        lo=lo,
        la=la,
        open_time=open_time,
        pic=pic
    )
    venues.append(venue)

    # if index == 4:
    #     break
    # print(index, org_type, poi_id, category, title, address, city, county, lo, la, open_time, pic)

In [9]:
# for venue in venues:
#     print(f"名称: {venue.open_time}")
#     print(f"地址: {venue.title}")
#     print(f"经度: {venue.address}")

名称: 周一至周日8:30-17：00
地址: 开欣社区妇女儿童活动中心
经度: 杭州市萧山区宁围街道开欣社区通惠北路与建设一路红路灯口雅棠轩小区1871-1873号
名称: 09:00-17:00
地址: 文昌镇王家源村文化礼堂
经度: 杭州市淳安县文昌镇王家源村
名称: 不开放
地址: 机房
经度: 杭州市丁桥景园南苑24号
名称: nan
地址: 公共数字文化服务点
经度: 杭州市西溪路618号2幢
名称: 周一至周五8:30-11:00；13:30-16:30
地址: 藕湖浜社区文化家园
经度: 杭州市萧山区城厢街道藕湖浜社区新桥园小区6幢


In [26]:
# import json
# 
# # 将这些对象转换为字典列表
# venues_data = [venue.to_dict() for venue in venues]
# 
# # 将字典列表写入 JSON 文件
# with open('venues.json', 'w', encoding='utf-8') as file:
#     json.dump(venues_data, file, ensure_ascii=False, indent=4)

In [33]:
# 写入 CSV 文件
import csv

with open('venues.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    # 写入 CSV 表头
    writer.writerow(["类型", "类别ID", "类别", "名称", "地址", "城市", "区县", "经度", "纬度", "开放时间", "图片"])
    for venue in venues:
        writer.writerow(
            [venue.org_type, venue.category_id, venue.category, venue.title, venue.address, venue.city, venue.county,
             venue.longitude, venue.latitude, venue.open_time, venue.picture])


In [ ]:
# 读csv文件
import csv

# 使用分号作为分隔符读取 CSV 文件
with open('venues.csv', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file, delimiter=';')
    venues_from_csv = []
    for row in reader:
        venue = VenueInfo(
            row["类型"],
            (row["类别ID"]),
            row["类别"],
            row["名称"],
            row["地址"],
            row["城市"],
            row["区县"],
            (row["经度"]),
            (row["纬度"]),
            row["开放时间"],
            row["图片"]
        )
        venues_from_csv.append(venue)

In [41]:
# 打印从 CSV 文件中读取的数据，前10条
for venue in venues_from_csv[:10]:
    print(venue.picture)
    # print(venue.__dict__)

/whyx20221018/99/7907.jpg
/whyx20221018/99/14914.jpg
/whyx20221018/98/7999.jpg
/whyx20221018/98/7962.jpg
/whyx20221018/98/7470.jpg
/whyx20221018/98/5168.jpg
/whyx20221018/98/10698.jpg
/whyx20221018/97/8670.jpg
/whyx20221018/97/5796.jpg
/whyx20221018/96/7528.jpg


In [63]:
import requests
import json

error_list = []


def process_picture(picture_url):
    print(picture_url)
    # API 请求的URL
    url = 'https://fxhz.wgly.hangzhou.gov.cn/dashboard/api/oss/transfer/2oss'

    # 请求的头部
    headers = {
        'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
        'Content-Type': 'application/json'
    }

    # 请求的正文
    payload = json.dumps({
        "urlList": [picture_url],
        "separator": ".cn/whyxoss/"
    })

    # 发送 POST 请求
    try:
        response = requests.post(url, headers=headers, data=payload)

        # 检查响应状态码
        if response.status_code == 200:
            # 假设返回的 JSON 中有一个 'data' 字段
            data = response.json().get('data')
            return json.loads(data)[0]
        else:
            error_list.append(picture_url)
            print(f"请求失败，状态码: {response.status_code}")
            return 'oooo'
    except Exception as e:
        error_list.append(picture_url)
        print(f"处理图片过程中发生异常: {e}")
        return 'oooo1'

In [48]:
# 写入 CSV 文件,追加

def write_csv(venues_with_result):
    with open('result.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        # 写入 CSV 表头
        writer.writerow(
            ["类型", "类别ID", "类别", "名称", "地址", "城市", "区县", "经度", "纬度", "开放时间", "图片", "img"])
        for venue in venues_with_result:
            writer.writerow(
                [venue['org_type'], venue['category_id'], venue['category'], venue['title'], venue['address'],
                 venue['city'], venue['county'], venue['longitude'], venue['latitude'], venue['open_time'],
                 venue['picture'], venue['img']]
            )


In [65]:
# 打印从 CSV 文件中读取的数据，前10条

# 定义一个列表来存储 venue 和新字段 result
venues_with_result = []
prefix = 'https://fxhz.wgly.hangzhou.gov.cn/whyxoss'

len(venues_from_csv)

# :1000
# 1000:2000
for venue in venues_from_csv[1000:2000]:
    # print(venue.__dict__)

    result = process_picture(prefix + venue.picture)
    # 创建一个包含 venue 信息和新 result 字段的元组
    venues_with_result.append({
        'org_type': venue.org_type,
        'category_id': venue.category_id,
        'category': venue.category,
        'title': venue.title,
        'address': venue.address,
        'city': venue.city,
        'county': venue.county,
        'longitude': venue.longitude,
        'latitude': venue.latitude,
        'open_time': venue.open_time,
        'picture': venue.picture,
        'img': result  # 新字段
    })

write_csv(venues_with_result)
venues_with_result = []

https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/99/7907.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/99/14914.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/98/7999.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/98/7962.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/98/7470.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/98/5168.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/98/10698.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/97/8670.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/97/5796.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/96/7528.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/96/15268.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/96/11415.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/95/8771.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/95/6210.jpg
https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/95/

In [66]:
print(error_list)
len(venues_from_csv)

['https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx/f780ce384afe4c76a8b2864fc7b77590.jpg', 'https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx/d3ea755512714fa59e08e8ece11dcecd.jpg', 'https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx/bf507ed675684ce3b9c436af3c37b16e.jpg', 'https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx/bae3ab8ca68e4ecfa8e74bc5dbaaa5a0.png', 'https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx/b2c1ed66f70e4d7f8f72c6a5a22df560.jpg', 'https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx/980eab38e51a45e7a0b6872234195683.jpg\n']


7531

In [45]:
a = process_picture('https://fxhz.wgly.hangzhou.gov.cn/whyxoss/whyx20221018/99/8007.jpg')
print(a)
print(error_list)

["https://fxhz.wgly.hangzhou.gov.cn/oss/whyx20221018/99/8007.jpg"]


In [59]:
a

'["https://fxhz.wgly.hangzhou.gov.cn/oss/whyx20221018/99/8007.jpg"]'

In [60]:
print(a)

["https://fxhz.wgly.hangzhou.gov.cn/oss/whyx20221018/99/8007.jpg"]


In [62]:
json.loads(a)[0]

'https://fxhz.wgly.hangzhou.gov.cn/oss/whyx20221018/99/8007.jpg'